# Part 6: AgentReview Demo - 5단계 Peer Review 시뮬레이션

**SNU AI Psychology Workshop - February 2026**

---

## AgentReview (EMNLP 2024)

실제 ICLR peer review 프로세스를 LLM으로 시뮬레이션하는 프레임워크입니다.

**5단계 파이프라인:**
1. **Reviewer Assessment**: 3명의 리뷰어가 독립적으로 논문 평가
2. **Author-Reviewer Discussion**: 저자가 rebuttal 제출
3. **Reviewer-AC Discussion**: Area Chair가 리뷰어 간 토론 중재
4. **Meta-Review**: AC가 종합 리뷰 작성
5. **Paper Decision**: Accept/Reject 최종 결정

**참고 자료:**
- 📄 [논문](https://arxiv.org/abs/2406.12708)
- 🌐 [웹사이트](https://agentreview.github.io/)
- 💻 [GitHub](https://github.com/Ahren09/AgentReview)
- 💡 [데모](https://huggingface.co/spaces/Ahren09/AgentReview)
- 📦 [Full 데이터 (Dropbox)](https://www.dropbox.com/scl/fi/l17brtbzsy3xwflqd58ja/AgentReview_Paper_Data.zip?rlkey=vldiexmgzi7zycmz7pumgbooc&st=b6g3nkry&dl=0)

---
## Setup

In [ ]:
# Cell 1: 환경 설정
import os
import sys

try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경")
except ImportError:
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
if IN_COLAB:
    print("📦 설치 중...")
    %pip install openai python-dotenv -q
    !git clone https://github.com/ahren09/agentreview.git 2>/dev/null || echo "✓ AgentReview 이미 존재"
    print("✅ 완료!")
else:
    print("✅ 로컬: pyproject.toml 사용")
    import subprocess
    if not os.path.exists("agentreview"):
        print("📥 AgentReview 클론 중...")
        subprocess.run(["git", "clone", "https://github.com/ahren09/agentreview.git"], capture_output=True)
    else:
        print("✓ AgentReview 이미 존재")

In [ ]:
# Cell 3: API Key
if IN_COLAB:
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        print("✅ Colab Secrets에서 로딩")
    except:
        OPENAI_API_KEY = None
        print("⚠️ Colab Secrets에 OPENAI_API_KEY 추가 필요")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
        if OPENAI_API_KEY:
            print("✅ .env 파일에서 로딩")
        else:
            print("⚠️ .env 파일에 OPENAI_API_KEY 추가 필요")
    except:
        OPENAI_API_KEY = None

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY or ''
print(f"\n🔑 OpenAI: {'설정됨' if OPENAI_API_KEY else '없음'}")

---
## 데이터 확인

In [ ]:
# Cell 4: 샘플 데이터 확인
import os

data_dir = "data/ICLR2022"

if os.path.exists(data_dir):
    print("✅ ICLR2022 샘플 데이터 발견!")
    print(f"\n📁 구조:")
    print(f"   {data_dir}/")
    print(f"   ├── paper/")
    print(f"   │   ├── Accept-oral/ (2개)")
    print(f"   │   └── Reject/ (2개)")
    print(f"   └── notes/ (메타데이터)")
    
    # 논문 ID 확인
    import json
    with open(f"{data_dir}/id2decision.json") as f:
        id2decision = json.load(f)
    
    print(f"\n📄 포함된 논문:")
    for pid, decision in id2decision.items():
        print(f"   - Paper {pid}: {decision}")
    
    USE_AGENTREVIEW = True
else:
    print("⚠️ 샘플 데이터 없음")
    print("\n이 노트북은 실제 AgentReview 데모용입니다.")
    print("간단한 리뷰는 notebook 4를 사용하세요.")
    USE_AGENTREVIEW = False

---
## 리뷰어 페르소나 설정

In [ ]:
# Cell 5: 리뷰어 설정

# 기본 설정: 모든 리뷰어가 일반적
baseline_setting = {
    "reviewer": ["BASELINE", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험: 악의적 리뷰어 1명
malicious_setting = {
    "reviewer": ["malicious", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험: 호의적 리뷰어 1명
benign_setting = {
    "reviewer": ["benign", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

print("✅ 리뷰어 페르소나 정의 완료")
print("\n사용 가능:")
print("  - BASELINE: 일반 리뷰어")
print("  - malicious: 악의적 (낮은 점수 선호)")
print("  - benign: 호의적 (높은 점수 선호)")

---
## AgentReview 실행

In [ ]:
# Cell 6: AgentReview 모듈 로딩
if USE_AGENTREVIEW and OPENAI_API_KEY:
    import sys
    sys.path.insert(0, os.path.abspath("agentreview"))
    
    try:
        from agentreview.environments import PaperReview
        from agentreview.paper_review_arena import PaperReviewArena
        from agentreview.paper_review_settings import get_experiment_settings
        from agentreview.utility.experiment_utils import initialize_players
        from agentreview.utility.utils import get_paper_decision_mapping
        
        print("✅ AgentReview 모듈 로딩 완료")
        READY = True
    except Exception as e:
        print(f"❌ 로딩 실패: {e}")
        READY = False
else:
    print("⚠️ 실행 불가:")
    if not USE_AGENTREVIEW:
        print("   - 데이터 없음")
    if not OPENAI_API_KEY:
        print("   - OpenAI API key 없음")
    READY = False

In [ ]:
# Cell 7: Demo 실행 (샘플 논문 2개)
if READY:
    from argparse import Namespace
    
    # 설정
    args = Namespace(
        openai_client_type='openai',
        conference='ICLR2022',  # 샘플 데이터 연도
        num_reviewers_per_paper=3,
        model_name='gpt-4o-mini',  # 비용 절감
        output_dir='outputs',
        data_dir='./data',
        acceptance_rate=0.32,
        skip_logging=False,
        task='paper_review'
    )
    
    # 논문 ID 매핑
    paper_id2decision, paper_decision2ids = get_paper_decision_mapping(
        args.data_dir, 
        args.conference
    )
    
    # 샘플 논문 2개 (Accept 1개 + Reject 1개)
    sample_ids = [284, 216]  # Accept, Reject
    
    print(f"📝 {len(sample_ids)}개 논문 리뷰 시작...\n")
    
    for paper_id in sample_ids:
        paper_decision = paper_id2decision[str(paper_id)]
        
        # 실험 설정 선택 (baseline vs malicious)
        # <- 여기서 변경: baseline_setting, malicious_setting, benign_setting
        experiment_setting = get_experiment_settings(
            paper_id=paper_id,
            paper_decision=paper_decision,
            setting=baseline_setting
        )
        
        print(f"\n{'='*60}")
        print(f"📄 Paper {paper_id}")
        print(f"   실제 결과: {paper_decision}")
        print(f"   리뷰어: {experiment_setting['reviewer']}")
        print(f"{'='*60}\n")
        
        # 플레이어 초기화
        players = initialize_players(
            experiment_setting=experiment_setting, 
            args=args
        )
        
        # 환경 생성
        env = PaperReview(
            player_names=[p.name for p in players],
            paper_decision=paper_decision,
            paper_id=paper_id,
            args=args,
            experiment_setting=experiment_setting
        )
        
        # Arena 실행
        arena = PaperReviewArena(
            players=players, 
            environment=env, 
            args=args
        )
        
        # 5단계 파이프라인 실행
        arena.launch_cli(interactive=False)
        
        print("\n✅ 리뷰 완료\n")
else:
    print("⚠️ 실행 불가 (위 셀 확인)")

---
## 결과 확인

In [ ]:
# Cell 8: 생성된 파일 확인
if READY and os.path.exists('outputs'):
    print("📊 생성된 리뷰:")
    print(f"   폴더: outputs/")
    print("\n5단계 파일:")
    print("  - Phase I: 리뷰어 평가")
    print("  - Phase II: Author rebuttal")
    print("  - Phase III: 리뷰어-AC 토론")
    print("  - Phase IV: Meta-review")
    print("  - Phase V: 최종 결정")
    
    files = os.listdir('outputs')
    print(f"\n📁 총 {len(files)}개 파일 생성")
    
    # 최근 파일 5개
    print("\n최근 파일:")
    for f in sorted(files)[-5:]:
        print(f"  - {f}")
else:
    print("결과 없음")

---
## DIY: 설정 비교 실험

In [ ]:
# DIY Cell 9: 다양한 설정 비교

# 실험 아이디어:
# 1. baseline vs malicious: 악의적 리뷰어의 영향?
#    -> Cell 7에서 malicious_setting으로 변경 후 재실행
#
# 2. baseline vs benign: 호의적 리뷰어의 영향?
#    -> Cell 7에서 benign_setting으로 변경 후 재실행
#
# 3. 모든 논문(4개) 테스트:
#    -> Cell 7의 sample_ids를 [284, 154, 216, 174]로 변경

# 비교 포인트:
# - 리뷰어 점수가 바뀌는가?
# - 최종 Accept/Reject 결정이 달라지는가?
# - Rebuttal이 효과가 있는가?

---
## 토론

### 질문

1. **리뷰어 페르소나의 영향**
   - malicious 리뷰어가 최종 결정을 바꾸나요?
   - 3명 중 1명만 달라도 영향이 큰가요?

2. **5단계 파이프라인의 가치**
   - Author rebuttal이 점수를 올리나요?
   - AC 토론이 편향을 줄이나요?

3. **실제 활용**
   - Peer review 시스템 개선에 어떻게 사용?
   - 논문 투고 전 사전 테스트?
   - 리뷰어 배정 최적화?